In [ ]:
# Based on docs:
# https://docs.lightly.ai/self-supervised-learning/tutorials/package/tutorial_simclr_clothing.html

# Also, see:
# https://github.com/giakoumoglou/classification/blob/main/notebooks/main_simclr.ipynb
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html

## Imports

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import os
from typing import Union, List, Tuple
from pathlib import Path
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from PIL import Image

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.preprocessing import normalize

from lightly.data import LightlyDataset
from lightly.transforms import SimCLRTransform, utils

from utils import (
    get_image_as_np_array,
    plot_knn_clusters,
    get_distance_between_points_in_cluster,
    get_distances_between_centroids,
    plot_clusters,
    generate_embeddings_and_fnames_simclr,
    check_labels_correspondence,
    plot_knn_examples,
    plot_clusters_3d,
    generate_embeddings,
    prepare_mnist_images,
)

## Configuration

We set some configuration parameters for our experiment.
Feel free to change them and analyze the effect.

The default configuration with a batch size of 256 and input resolution of 128
requires 6GB of GPU memory.



In [ ]:
# DEBUG
# device = "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

accelerator = "gpu" if device == "cuda" else "cpu"

print(f"Using device: {device}")
print(f"Using accelerator: {accelerator}")

In [ ]:
# input_size = 32  # laptop
input_size = 128  # PC
# batch_size = 64  # laptop
batch_size = 256  # PC

num_workers = 8
seed = 1
max_epochs = 100
num_ftrs = 32

path_to_data = Path("datasets/MNIST")
path_to_train_data = path_to_data / "train"
path_to_test_data = path_to_data / "test"

Let's set the seed for our experiments



In [ ]:
pl.seed_everything(seed)

## Data preparation

We make sure MNIST dataset is downloaded and save only selected classes to .jpg files.
This is to ensure to shuffiling problem arrises after unsupervised part.

In [ ]:
prepare_mnist_images([0, 4, 9], path_to_data)

## Setup data augmentations and loaders

We prepare data augmentations and loaders for unsupervised training, using lightly classes, like `SimCLRTransform` and `LightlyDataset`.

### Train

In [ ]:
train_transform = SimCLRTransform(input_size=input_size, vf_prob=0.5, rr_prob=0.5)

In [ ]:
train_dataset_simclr = LightlyDataset(
    input_dir=path_to_train_data, transform=train_transform
)

In [ ]:
print(len(train_dataset_simclr))

In [ ]:
sample, target, fname = train_dataset_simclr[17111]
print("sample", sample)
print("target", target)
print("fname", fname)

# targets got acquired weirdly

In [ ]:
dataloader_train_simclr = torch.utils.data.DataLoader(
    train_dataset_simclr,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
)

In [ ]:
train_example_idx = 65

In [ ]:
for batch in dataloader_train_simclr:
    print("\ngeneral batch info")
    print(type(batch))
    print(len(batch))
    print(type(batch[0]))
    print(len(batch[0]))

    print("\nimage batch info")
    print(type(batch[0][0]))
    print(batch[0][0].shape)
    print(batch[0][1].shape)
    plt.imshow(batch[0][0][train_example_idx].permute(1, 2, 0))

    print("\ntarget")
    print(type(batch[1][train_example_idx]))
    print(batch[1][train_example_idx])

    print("\nfilename")
    print(type(batch[2][train_example_idx]))
    print(batch[2][train_example_idx])
    break

### Test

In [ ]:
test_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((input_size, input_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=utils.IMAGENET_NORMALIZE["mean"],
            std=utils.IMAGENET_NORMALIZE["std"],
        ),
    ]
)

In [ ]:
test_dataset_simclr = LightlyDataset(
    input_dir=path_to_test_data, transform=test_transform
)

In [ ]:
print(len(test_dataset_simclr))

In [ ]:
sample, target, fname = test_dataset_simclr[1111]
print("sample", sample)
print("target", target)
print("fname", fname)
# so the targets are correct here - they are just sequential, instead of infered from dir names

In [ ]:
dataloader_test_simclr = torch.utils.data.DataLoader(
    test_dataset_simclr,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True,
    num_workers=num_workers,
)

In [ ]:
for batch in dataloader_test_simclr:
    print("\ngeneral batch info")
    print(type(batch))
    print(len(batch))
    print(type(batch[0]))
    print(len(batch[0]))

    print("\nimage batch info")
    print(type(batch[0][0]))
    print(batch[0][33].shape)
    print(batch[0][33].shape)
    plt.imshow(batch[0][33].permute(1, 2, 0))

    print("\ntarget")
    print(type(batch[1][33]))
    print(batch[1][33])

    print("\nfilename")
    print(type(batch[2][33]))
    print(batch[2][33])
    break

## Create the SimCLR Model
Now we create the SimCLR model. We implement it as a PyTorch Lightning Module
and use a ResNet-18 backbone from Torchvision. Lightly provides implementations
of the SimCLR projection head and loss function in the `SimCLRProjectionHead`
and `NTXentLoss` classes. We can simply import them and combine the building
blocks in the module.



In [ ]:
from lightly.loss import NTXentLoss
from lightly.models.modules.heads import SimCLRProjectionHead


class SimCLRModel(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # create a ResNet backbone and remove the classification head
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])

        hidden_dim = resnet.fc.in_features
        self.projection_head = SimCLRProjectionHead(hidden_dim, hidden_dim, 64)
        # output_dim = 128 -> embeddings.shape = 512
        # output_dim = 64 -> embeddings.shape = 512
        # because backbone output is 512, and this works every time
        self.criterion = NTXentLoss()

    def forward(self, x):
        h = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(h)
        return z

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]

Train the module using the PyTorch Lightning Trainer on a single GPU.



In [ ]:
model = SimCLRModel()

In [ ]:
early_stopping_callback = EarlyStopping(
    monitor="train_loss_ssl", mode="min", patience=5
)

model_checkpoint_callback = ModelCheckpoint(
    monitor="train_loss_ssl", mode="min", save_top_k=-1
)

### Train

In [ ]:
trainer = pl.Trainer(
    log_every_n_steps=10,
    max_epochs=4,
    # max_epochs=max_epochs,
    devices=1,
    accelerator=accelerator,
    callbacks=[early_stopping_callback, model_checkpoint_callback],
    logger=CSVLogger(save_dir="lightning_logs", name="simclr_mnist"),
)

In [ ]:
# trainer.fit(model, dataloader_train_simclr)

Next we create a helper function to generate embeddings
from our test images using the model we just trained.
Note that only the backbone is needed to generate embeddings,
the projection head is only required for the training.
Make sure to put the model into eval mode for this part!



### Eval

In [ ]:
# For eval only

# PC-trained model
# model = SimCLRModel.load_from_checkpoint(
#     "./lightning_logs/old_stuff/version_38/checkpoints/epoch=16-step=1173.ckpt"
# )

model = SimCLRModel.load_from_checkpoint(
    "./lightning_logs/simclr_mnist/version_4/checkpoints/epoch=3-step=276.ckpt"
)

# laptop-trained model
# maybe not-OK, as it was trained on 70k images (both train and test)
# model = SimCLRModel.load_from_checkpoint(
#     "./lightning_logs/version_0/checkpoints/epoch=9-step=10930.ckpt"
# )

In [ ]:
model.eval()

In [ ]:
# let's use train dataloader for now
embeddings, filenames = generate_embeddings(model, dataloader_test_simclr)

In [ ]:
print("embeddings")
print(type(embeddings))
print(embeddings.shape)

print("filenames")
print(type(filenames))
print(len(filenames))

In [ ]:
print(embeddings[0].max())
print(embeddings[0].min())
print(embeddings[0].mean())
print(embeddings[0].std())

In [ ]:
print(embeddings[0].histogram())
hist, bin_edges = embeddings[0].histogram(bins=100)
plt.bar(bin_edges[:-1], hist, width=0.5)

In [ ]:
print(filenames[0])

## Clustering
Will basically consist of 2 steps:
1. Dimensionality reduction (UMAP, PCA or t-SNE)
2. Visualization

### Dimensionality reduction

In [ ]:
pca = PCA(n_components=50)
data_pca = pca.fit_transform(embeddings.cpu().numpy())

tsne = TSNE(n_components=2)
data_tsne = tsne.fit_transform(data_pca)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
clusters_tsne = kmeans.fit_predict(data_tsne)
clusters_pca = kmeans.fit_predict(data_pca)

unique, counts = np.unique(clusters_tsne, return_counts=True)
print("TSNE")
print(dict(zip(unique, counts)))

unique, counts = np.unique(clusters_pca, return_counts=True)
print("PCA")
print(dict(zip(unique, counts)))

In [ ]:
filenames[0].split("\\")

In [ ]:
targets = [int(fname.split("\\")[0]) for fname in filenames]  # windows
print(len(targets))
print(targets[:10])
print(targets[1300:1310])
print(targets[-10:])

In [ ]:
plt.scatter(data_tsne[:, 0], data_tsne[:, 1], c=targets, cmap="viridis")
plt.title("t-SNE visualization")
plt.colorbar()
plt.show()

In [ ]:
plt.scatter(data_pca[:, 48], data_pca[:, 49], c=targets, cmap="viridis")
plt.title("PCA visualization")
plt.colorbar()
plt.show()